In [ ]:
%load_ext autoreload
%autoreload 2

from style_transfer import *
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import PIL
import numpy as np
from scipy.misc import imread
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from dataset import CocoStuffDataSet

In [ ]:
HEIGHT = WIDTH = 256
val_dataset = CocoStuffDataSet(mode='val', supercategories=['animal'], height=HEIGHT, width=WIDTH, do_normalize=False)


In [ ]:
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
cnn = torchvision.models.vgg16(pretrained=True).features
cnn.cuda()
# cnn.type(dtype)
# We don't want to train the model any further, so we don't want PyTorch to waste computation 
# computing gradients on parameters we're never going to update.
for param in cnn.parameters():
    param.requires_grad = False

In [ ]:
def get_style_weights(beta):
    style_weights = np.array([100, 100, 10, 10, 1])
    return [float(x) for x in beta*style_weights]


In [ ]:
def get_savename(content_idx, style_background, style_foreground=None):
    name = "{}_{}".format(content_idx, style_background.split('.')[0])
    if style_foreground:
        name += "_and_{}".format(style_foreground.split('.')[0])
    name += ".png"
    return name

In [ ]:
alpha = 5e-4
beta = 1e3
gamma = 1e-2
savedir = './saved_style_transfers'
if not os.path.exists(savedir):
    os.makedirs(savedir)
style_dir = '../styles/'


In [ ]:
backgrounds = ['the_scream.jpg', 'starry_night.jpg', 'composition_vii.jpg', 'tsunami.jpg']
foregrounds = ['van_gogh_portrait.jpg', 'guernica.jpg', 'guitar.jpg', 'muse.jpg']
background_images = [PIL.Image.open(os.path.join(style_dir, n)) for n in backgrounds]
foreground_images = [PIL.Image.open(os.path.join(style_dir, n)) for n in foregrounds]
for style_background_image in background_images:
    for style_foreground_image in foreground_images:
        # suggested indices: 417, 77, 1011, 913, 55
        for idx in (417, 77, 55):
            savename = get_savename(idx, style_background_name, style_foreground_name)
            savepath = os.path.join(savedir, savename)
            content_image, background_mask = get_image(val_dataset, idx)
            foreground_mask = 1.0 - background_mask

            transfer_params = {
                'cnn' : cnn,
                'content_image' : content_image,
                'style_image' : style_background_image,
                'content_mask': background_mask,
                'image_size' : HEIGHT,
                'content_layer' : 12,
                'content_weight' : alpha,
                'style_layers' : style_layers,
                'style_weights' : get_style_weights(beta),
                'tv_weight' : gamma,
                'max_iters' : 1000,
                'init_random' : False,
                'mask_layer' : True,
                'second_style_image' : style_foreground_image 
            }

            final_img, final_loss = style_transfer(**transfer_params)
            display_style_transfer(final_img, savepath)